In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
!pip install backtesting

In [ ]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    ns = 5 # 短期移動平均日数
    nl = 25 # 長期移動平均日数

    def init(self):
        # 短期移動平均
        self.smaS = self.I(SMA, self.data["Close"], self.ns) 
        # 長期移動平均 
        self.smaL = self.I(SMA, self.data["Close"], self.nl)
        
    def next(self):
        # smaS > smaL で買う
        if crossover(self.smaS, self.smaL):  
            self.buy() 
        # smaS < smaL で売る
        elif crossover(self.smaL, self.smaS): 
            self.position.close() 

In [ ]:
from backtesting import Backtest
import datetime as dt

# 新日本科学 （2395）の2021年1月〜2022年3月のデータ
df = get_stock_data(2395)
data = df[dt.datetime(2021,1,1):dt.datetime(2022,3,31)]

bt = Backtest(data, SmaCross, trade_on_close=True)

# バックテストを実行
result = bt.run() 
# 実行結果のデータを表示
print(result) 
# 実行結果をグラフで表示
bt.plot()

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   57.894737
Equity Final [$]                     16774.22
Equity Peak [$]                      23730.42
Return [%]                            67.7422
Buy & Hold Return [%]               156.44515
Return (Ann.) [%]                    53.53828
Volatility (Ann.) [%]               60.831364
Sharpe Ratio                          0.88011
Sortino Ratio                        2.202463
Calmar Ratio                         1.641885
Max. Drawdown [%]                  -32.607809
Avg. Drawdown [%]                   -6.024895
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                       65.26924
Worst Trade [%]                    -16.203137
Avg. Trade [%]                    

Row(id='1433', ...)

短期線と長期線の日数を最適化

In [ ]:
result = bt.optimize(ns=range(5, 25, 5),nl=range(5, 75, 5), maximize="Return [%]", constraint=lambda r: r.ns < r.nl)
print(result)
bt.plot()

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   57.894737
Equity Final [$]                     22927.87
Equity Peak [$]                      23581.51
Return [%]                           129.2787
Buy & Hold Return [%]               156.44515
Return (Ann.) [%]                     98.9402
Volatility (Ann.) [%]               80.428332
Sharpe Ratio                         1.230166
Sortino Ratio                        4.196628
Calmar Ratio                         3.977972
Max. Drawdown [%]                  -24.872023
Avg. Drawdown [%]                   -6.134172
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       24 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      74.131738
Worst Trade [%]                     11.320916
Avg. Trade [%]                    

Row(id='2865', ...)

RSIを使ったバックテスト

In [ ]:
import talib as ta

def RSI(close, n1, n2):
    rsiS = ta.RSI(close, timeperiod=n1)
    rsiL = ta.RSI(close, timeperiod=n2)
    return rsiS, rsiL

In [ ]:
class RSICross(Strategy):
    ns = 14 # 短期
    nl = 28 # 長期

    def init(self):
        self.rsiS, self.rsiL = self.I(RSI, self.data["Close"], self.ns, self.nl)

    def next(self):
        # 短期 >　長期で買い
        if crossover(self.rsiS, self.rsiL): 
            self.buy() 
        # 長期 > 短期で売り
        elif crossover(self.rsiL, self.rsiS):
            self.position.close()

In [ ]:
# 新日本科学 （2395）の2021年1月〜2022年3月のデータ
df = get_stock_data(2395)
data = df[dt.datetime(2021,1,1):dt.datetime(2022,3,31)]

bt = Backtest(data, RSICross, trade_on_close=True)

result = bt.run()
print(result)
bt.plot()

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   46.052632
Equity Final [$]                     26822.45
Equity Peak [$]                      27203.25
Return [%]                           168.2245
Buy & Hold Return [%]               156.44515
Return (Ann.) [%]                  126.570132
Volatility (Ann.) [%]               88.475086
Sharpe Ratio                         1.430574
Sortino Ratio                        7.913813
Calmar Ratio                         9.808338
Max. Drawdown [%]                   -12.90434
Avg. Drawdown [%]                   -3.918127
Max. Drawdown Duration       98 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   16
Win Rate [%]                            56.25
Best Trade [%]                      73.871413
Worst Trade [%]                     -3.977934
Avg. Trade [%]                    

Row(id='2192', ...)

RSI最適化

In [ ]:
result = bt.optimize(ns = range(5, 25, 5),
                     nl= range(5, 75, 5), 
                      maximize = "Return [%]",
                      constraint = lambda r: r.ns < r.nl)

print(result)
bt.plot()

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   50.328947
Equity Final [$]                     30669.04
Equity Peak [$]                      30669.04
Return [%]                           206.6904
Buy & Hold Return [%]               156.44515
Return (Ann.) [%]                  153.191333
Volatility (Ann.) [%]              102.278314
Sharpe Ratio                         1.497789
Sortino Ratio                        9.094367
Calmar Ratio                          8.60006
Max. Drawdown [%]                  -17.812821
Avg. Drawdown [%]                   -3.902784
Max. Drawdown Duration       94 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   23
Win Rate [%]                        47.826087
Best Trade [%]                      73.871413
Worst Trade [%]                     -4.215555
Avg. Trade [%]                    

Row(id='3006', ...)

MACDを使ったバックテスト

In [ ]:
def MACD(close, n1, n2, n3):
    macd, macdsignal, _ = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=n3)
    return macd, macdsignal

In [ ]:
class MACDCross(Strategy):
    n1 = 12
    n2 = 26
    n3 = 9

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, self.data["Close"], self.n1, self.n2, self.n3)

    def next(self): 
        # macd >　シグナルで買い
        if crossover(self.macd, self.macdsignal): 
            self.buy()
        # シグナル > macdで売り
        elif crossover(self.macdsignal, self.macd):
            self.position.close() 

In [ ]:
# 新日本科学 （2395）の2021年1月〜2022年3月のデータ
df = get_stock_data(2395)
data = df[dt.datetime(2021,1,1):dt.datetime(2022,3,31)]

bt = Backtest(data, MACDCross, trade_on_close=True)

result = bt.run()
print(result)
bt.plot()

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   48.355263
Equity Final [$]                     23972.46
Equity Peak [$]                      28863.74
Return [%]                           139.7246
Buy & Hold Return [%]               156.44515
Return (Ann.) [%]                  106.424748
Volatility (Ann.) [%]                82.13051
Sharpe Ratio                           1.2958
Sortino Ratio                        5.923195
Calmar Ratio                         5.278125
Max. Drawdown [%]                  -20.163361
Avg. Drawdown [%]                   -4.866328
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   10
Win Rate [%]                             60.0
Best Trade [%]                      86.666957
Worst Trade [%]                     -8.613144
Avg. Trade [%]                    

Row(id='3102', ...)

In [ ]:
result = bt.optimize(n1=range(5, 75, 5), n2=range(10, 75, 5), n3=range(10, 75, 5), maximize = "Return [%]" , constraint = lambda r: r.n1 < r.n2) 

print(result)
bt.plot()

/usr/local/lib/python3.7/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1183 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   47.697368
Equity Final [$]                     30032.08
Equity Peak [$]                      30368.04
Return [%]                           200.3208
Buy & Hold Return [%]               156.44515
Return (Ann.) [%]                  148.824518
Volatility (Ann.) [%]              100.970686
Sharpe Ratio                         1.473938
Sortino Ratio                        8.594108
Calmar Ratio                         9.193035
Max. Drawdown [%]                  -16.188834
Avg. Drawdown [%]                   -4.224069
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                      77.419565
Worst Trade [%]                    -10.353969
Avg. Trade [%]                    

Row(id='2289', ...)